In [ ]:
## Pseudo-bulk DE analysis of multiome subjects
## Andi Liu
# 5/26/2025

library(Libra)
library(Seurat)
library(pbmcapply)

library(dplyr)# for dataframe processing
library(tidyr)#
library(tidydr)
library(magrittr) # for %<>%

library(ggplot2)
library(pheatmap)

In [ ]:
## load previouse object
rm(list = ls())
gc()
object <- readRDS("cellbender_HIP_object_10.31.rds")
object

## Run Pseudo-bulk DE analysis on selected cell type

In [ ]:
## Create a wrapper function for pseudo-bulk DE analysis
pseudo_bulk_DE <- function(object, celltype, min_features_pct) {
    # Filter the object for the specified cell type
    obj <- subset(object,subset = cluster_celltype == celltype)
    print(celltype)

    ## get the list of genes for analysis
    expr <- LayerData(obj,assay = "PC",layer = "counts")
    meta <- obj@meta.data
    # check minimum features
    genes.percent.expression <- rowMeans(expr>0)
    keep <- names(genes.percent.expression[genes.percent.expression>min_features_pct])
    print(length(keep))

    # pseudobulk the counts based on donor-condition-celltype
    pseudo_object <- AggregateExpression(obj, assays = "PC", return.seurat = T, group.by = c("individual_ID","diagnosis"))
    pseudo_object

    ## Running DESeq2
    Idents(pseudo_object) <- "diagnosis"

    bulk_de <- FindMarkers(object = pseudo_object, 
                            ident.1 = "EOAD", 
                            ident.2 = "NCI",
                            test.use = "DESeq2",
                            features = keep
                            )

    ## Add gene names and cell type
    bulk_de$gene  <- rownames(bulk_de)
    bulk_de$celltype <- celltype
    ## Filter results with NA values in p_val_adj
    bulk_de <- bulk_de %>% 
      filter(!is.na(p_val_adj)) %>%
      arrange(p_val_adj)
    ## Show top results
    head(bulk_de)
    
    ## return the results
    return(bulk_de)
}

## Run the pseudo-bulk DE analysis on HIP object

In [ ]:
res_all  <- data.frame()
for (i in unique(object$cluster_celltype)) {
    print(i)
    # run the pseudo-bulk DE analysis
    result <- pseudo_bulk_DE(object, celltype = i, min_features_pct = 0.25)

    # Add the results to the combined dataframe
    res_all <- rbind(res_all, result)
}

# Save the results to a CSV file
res_all$region <- "HIP"
write.csv(res_all,"./Results/Revision/HIP_pseudobulk_DE_results.csv", row.names = F)

In [ ]:
res_hip <- read.csv("./Results/DEG/Overlap_mast_mixed_HIP.csv",row.names = 1)
head(res_hip)

In [ ]:
# ## Organize the results
# res_all_sig <- res_all[res_all$p_val_adj <= 0.05 & abs(res_all$avg_log2FC) >= 0.25,]
# res_all_sig$dir <- ifelse(res_all_sig$avg_log2FC > 0, "pos", "neg")
# res_all_sig$comb <- paste(res_all_sig$gene,res_all_sig$dir,res_all_sig$celltype, sep = "_")

# ## Check if consistent with previous results
# res_all_sig$in_overlap <- ifelse(res_all_sig$comb %in% res_hip$comb, "yes", "no")
# table(res_all_sig$celltype, res_all_sig$in_overlap)

In [ ]:
# ## save the results
# write.csv(res_all_sig,"./Results/Revision/HIP_pseudobulk_DE_results_sig.csv", row.names = F)

## running the DE analysis on PFC object

In [ ]:
## running the DE analysis on PFC object
rm(list = ls())
gc()
object <- readRDS("cellbender_PFC_object_10.31.rds")
object

In [ ]:
res_all  <- data.frame()
for (i in unique(object$cluster_celltype)) {
    print(i)
    # run the pseudo-bulk DE analysis
    result <- pseudo_bulk_DE(object, celltype = i, min_features_pct = 0.25)

    # Add the results to the combined dataframe
    res_all <- rbind(res_all, result)
}

# Save the results to a CSV file
res_all$region <- "PFC"
write.csv(res_all,"./Results/Revision/PFC_pseudobulk_DE_results.csv", row.names = F)

In [ ]:
res_pfc <- read.csv("./Results/DEG/Overlap_mast_mixed_PFC.csv",row.names = 1)
head(res_pfc)

In [ ]:
# ## Organize the results
# res_all_sig <- res_all[res_all$p_val_adj < 0.05 & res_all$avg_log2FC > 0.25,]
# res_all_sig$dir <- ifelse(res_all_sig$avg_log2FC > 0, "pos", "neg")
# res_all_sig$comb <- paste(res_all_sig$gene,res_all_sig$dir,res_all_sig$celltype, sep = "_")

# ## Check if consistent with previous results
# res_all_sig$in_overlap <- ifelse(res_all_sig$comb %in% res_pfc$comb, "yes", "no")
# table(res_all_sig$celltype, res_all_sig$in_overlap)

In [ ]:
# ## save the results
# write.csv(res_all_sig,"./Results/Revision/PFC_pseudobulk_DE_results_sig.csv", row.names = F)

## running the DE analysis on EC object

In [ ]:
# rm(list = ls())
gc()
object <- readRDS("cellbender_EC_object_10.31.rds")
object

In [ ]:
res_all  <- data.frame()
for (i in unique(object$cluster_celltype)) {
    print(i)
    # run the pseudo-bulk DE analysis
    result <- pseudo_bulk_DE(object, celltype = i, min_features_pct = 0.25)

    # Add the results to the combined dataframe
    res_all <- rbind(res_all, result)
}

# Save the results to a CSV file
res_all$region <- "EC"
write.csv(res_all,"./Results/Revision/EC_pseudobulk_DE_results.csv", row.names = F)

In [ ]:
res_ec <- read.csv("./Results/DEG/Overlap_mast_mixed_EC.csv",row.names = 1)
head(res_ec)

In [ ]:
# ## Organize the results
# res_all_sig <- res_all[res_all$p_val_adj < 0.05 & res_all$avg_log2FC > 0.25,]
# res_all_sig$dir <- ifelse(res_all_sig$avg_log2FC > 0, "pos", "neg")
# res_all_sig$comb <- paste(res_all_sig$gene,res_all_sig$dir,res_all_sig$celltype, sep = "_")

# ## Check if consistent with previous results
# res_all_sig$in_overlap <- ifelse(res_all_sig$comb %in% res_ec$comb, "yes", "no")
# table(res_all_sig$celltype, res_all_sig$in_overlap)

In [ ]:
# ## save the results
# write.csv(res_all_sig,"./Results/Revision/EC_pseudobulk_DE_results_sig.csv", row.names = F)

## Summaryize all the results

In [ ]:
pseudobulk_PFC <- read.csv("./Results/Revision/PFC_pseudobulk_DE_results.csv")
pseudobulk_EC <- read.csv("./Results/Revision/EC_pseudobulk_DE_results.csv")
pseudobulk_HIP <- read.csv("./Results/Revision/HIP_pseudobulk_DE_results.csv")

In [ ]:
pseudobulk_PFC = pseudobulk_PFC %>%
  filter(p_val_adj <= 0.05 & abs(avg_log2FC) >= 0.25)
pseudobulk_EC = pseudobulk_EC %>%   
  filter(p_val_adj <= 0.05 & abs(avg_log2FC) >= 0.25)
pseudobulk_HIP = pseudobulk_HIP %>%   
  filter(p_val_adj <= 0.05 & abs(avg_log2FC) >= 0.25)   

## provide the direction of the log2FC
pseudobulk_PFC$dir = ifelse(pseudobulk_PFC$avg_log2FC > 0, "pos", "neg")
pseudobulk_EC$dir = ifelse(pseudobulk_EC$avg_log2FC > 0, "pos", "neg")
pseudobulk_HIP$dir = ifelse(pseudobulk_HIP$avg_log2FC > 0, "pos", "neg")

In [ ]:
### load the results from the previous analysis
overlap_PFC = read.csv("./Results/DEG/Overlap_mast_mixed_PFC.csv",row.names = 1)
overlap_EC = read.csv("./Results/DEG/Overlap_mast_mixed_EC.csv",row.names = 1)
overlap_HIP = read.csv("./Results/DEG/Overlap_mast_mixed_HIP.csv",row.names = 1)

In [ ]:
## working on the PFC results
df1 = pseudobulk_PFC %>% 
    select(gene,celltype1 = celltype, dir) %>%
    mutate(label1 = paste0(celltype1, "_", dir))
df2 = overlap_PFC %>% 
    select(gene,celltype2 = celltype, dir) %>%
    mutate(label2 = paste0(celltype2, "_", dir))

# Find overlapping genes (same gene and dir)
overlap_df <- inner_join(df1, df2, by = c("gene", "dir"),relationship = "many-to-many")

# Count overlaps (label1 x label2)
overlap_counts <- overlap_df %>%
  distinct(gene, label1, label2) %>%
  count(label1, label2, name = "overlap")

# Count total genes in df2 by label2
total_df2 <- df2 %>%
  distinct(gene, label2) %>%
  count(label2, name = "total_df2")

# Merge and calculate fraction
annotated <- left_join(overlap_counts, total_df2, by = "label2") %>%
  mutate(fraction = overlap / total_df2,
         label = sprintf("%d (%.2f)", overlap, fraction))

# Wide format for matrix of labels
label_mat_df <- annotated %>%
  select(label1, label2, label) %>%
  pivot_wider(names_from = label2, values_from = label, values_fill = "")

# Wide format for numeric matrix for coloring
value_mat_df <- annotated %>%
  select(label1, label2, overlap) %>%
  pivot_wider(names_from = label2, values_from = overlap, values_fill = 0)

# Build matrices
label_mat <- as.matrix(label_mat_df[,-1])
rownames(label_mat) <- label_mat_df$label1

value_mat <- as.matrix(value_mat_df[,-1])
rownames(value_mat) <- value_mat_df$label1

# Optional: restrict to shared labels
common_labels <- intersect(rownames(value_mat), colnames(value_mat))
value_mat <- value_mat[common_labels, common_labels]
label_mat <- label_mat[common_labels, common_labels]

# Plot with counts and fractions
p1 =pheatmap(value_mat,
         cluster_rows = FALSE, cluster_cols = FALSE,
         display_numbers = label_mat,
         main = "DEGs in MAST+Mixed models overlapping with pseudo-bulk DEGs in PFC",
         color = colorRampPalette(c("white", "darkgreen"))(100))

In [ ]:
pseudobulk_PFC$comb = paste(pseudobulk_PFC$gene, pseudobulk_PFC$dir,pseudobulk_PFC$celltype, sep = "_")

## provide label to indicate if the gene in the original study was supported by the pseudo-bulk DE analysis
overlap_PFC$in_pseudobulk <- ifelse(overlap_PFC$comb %in% pseudobulk_PFC$comb, "yes", "no")
table(overlap_PFC$celltype, overlap_PFC$in_pseudobulk)
write.csv(overlap_PFC,"./Results/Revision/PFC_overlap_pseudobulk_DE_results.csv", row.names = F)

In [ ]:
## working on the EC results
df1 = pseudobulk_EC %>% 
    select(gene,celltype1 = celltype, dir) %>%
    mutate(label1 = paste0(celltype1, "_", dir))
df2 = overlap_EC %>% 
    select(gene,celltype2 = celltype, dir) %>%
    mutate(label2 = paste0(celltype2, "_", dir))

# Find overlapping genes (same gene and dir)
overlap_df <- inner_join(df1, df2, by = c("gene", "dir"),relationship = "many-to-many")

# Count overlaps (label1 x label2)
overlap_counts <- overlap_df %>%
  distinct(gene, label1, label2) %>%
  count(label1, label2, name = "overlap")

# Count total genes in df2 by label2
total_df2 <- df2 %>%
  distinct(gene, label2) %>%
  count(label2, name = "total_df2")

# Merge and calculate fraction
annotated <- left_join(overlap_counts, total_df2, by = "label2") %>%
  mutate(fraction = overlap / total_df2,
         label = sprintf("%d (%.2f)", overlap, fraction))

# Wide format for matrix of labels
label_mat_df <- annotated %>%
  select(label1, label2, label) %>%
  pivot_wider(names_from = label2, values_from = label, values_fill = "")

# Wide format for numeric matrix for coloring
value_mat_df <- annotated %>%
  select(label1, label2, overlap) %>%
  pivot_wider(names_from = label2, values_from = overlap, values_fill = 0)

# Build matrices
label_mat <- as.matrix(label_mat_df[,-1])
rownames(label_mat) <- label_mat_df$label1

value_mat <- as.matrix(value_mat_df[,-1])
rownames(value_mat) <- value_mat_df$label1

# Optional: restrict to shared labels
common_labels <- intersect(rownames(value_mat), colnames(value_mat))
value_mat <- value_mat[common_labels, common_labels]
label_mat <- label_mat[common_labels, common_labels]

# Plot with counts and fractions
p2 =pheatmap(value_mat,
         cluster_rows = FALSE, cluster_cols = FALSE,
         display_numbers = label_mat,
         main = "DEGs in MAST+Mixed models overlapping with pseudo-bulk DEGs in EC",
         color = colorRampPalette(c("white", "darkgreen"))(100))

In [ ]:
pseudobulk_EC$comb = paste(pseudobulk_EC$gene, pseudobulk_EC$dir,pseudobulk_EC$celltype, sep = "_")

## provide label to indicate if the gene in the original study was supported by the pseudo-bulk DE analysis
overlap_EC$in_pseudobulk <- ifelse(overlap_EC$comb %in% pseudobulk_EC$comb, "yes", "no")
table(overlap_EC$celltype, overlap_EC$in_pseudobulk)
write.csv(overlap_EC,"./Results/Revision/EC_overlap_pseudobulk_DE_results.csv", row.names = F)

In [ ]:
## working on the EC results
df1 = pseudobulk_HIP %>% 
    select(gene,celltype1 = celltype, dir) %>%
    mutate(label1 = paste0(celltype1, "_", dir))
df2 = overlap_HIP %>% 
    select(gene,celltype2 = celltype, dir) %>%
    mutate(label2 = paste0(celltype2, "_", dir))

# Find overlapping genes (same gene and dir)
overlap_df <- inner_join(df1, df2, by = c("gene", "dir"),relationship = "many-to-many")

# Count overlaps (label1 x label2)
overlap_counts <- overlap_df %>%
  distinct(gene, label1, label2) %>%
  count(label1, label2, name = "overlap")

# Count total genes in df2 by label2
total_df2 <- df2 %>%
  distinct(gene, label2) %>%
  count(label2, name = "total_df2")

# Merge and calculate fraction
annotated <- left_join(overlap_counts, total_df2, by = "label2") %>%
  mutate(fraction = overlap / total_df2,
         label = sprintf("%d (%.2f)", overlap, fraction))

# Wide format for matrix of labels
label_mat_df <- annotated %>%
  select(label1, label2, label) %>%
  pivot_wider(names_from = label2, values_from = label, values_fill = "")

# Wide format for numeric matrix for coloring
value_mat_df <- annotated %>%
  select(label1, label2, overlap) %>%
  pivot_wider(names_from = label2, values_from = overlap, values_fill = 0)

# Build matrices
label_mat <- as.matrix(label_mat_df[,-1])
rownames(label_mat) <- label_mat_df$label1

value_mat <- as.matrix(value_mat_df[,-1])
rownames(value_mat) <- value_mat_df$label1

# Optional: restrict to shared labels
common_labels <- intersect(rownames(value_mat), colnames(value_mat))
value_mat <- value_mat[common_labels, common_labels]
label_mat <- label_mat[common_labels, common_labels]

# Plot with counts and fractions
p3 =pheatmap(value_mat,
         cluster_rows = FALSE, cluster_cols = FALSE,
         display_numbers = label_mat,
         main = "DEGs in MAST+Mixed models overlapping with pseudo-bulk DEGs in HIP",
         color = colorRampPalette(c("white", "darkgreen"))(100))

In [ ]:
pseudobulk_HIP$comb = paste(pseudobulk_HIP$gene, pseudobulk_HIP$dir,pseudobulk_HIP$celltype, sep = "_")

## provide label to indicate if the gene in the original study was supported by the pseudo-bulk DE analysis
overlap_HIP$in_pseudobulk <- ifelse(overlap_HIP$comb %in% pseudobulk_HIP$comb, "yes", "no")
table(overlap_HIP$celltype, overlap_HIP$in_pseudobulk)
write.csv(overlap_HIP,"./Results/Revision/HIP_overlap_pseudobulk_DE_results.csv", row.names = F)

In [ ]:
pdf("./Results/Revision/overlap_DEGs_PFC.pdf")
print(p1)
dev.off()

pdf("./Results/Revision/overlap_DEGs_EC.pdf")
print(p2)
dev.off()

pdf("./Results/Revision/overlap_DEGs_HIP.pdf")
print(p3)
dev.off()

### Testing the interaction term of diagnosis and brain region using the overall object.
### Fitting a LRT model with and without interaction terms (brain regions)

In [ ]:
## load DESeq2 library
library(DESeq2)
library(RColorBrewer)

In [ ]:
object <- readRDS("03.clean_object.10.31.rds")

DefaultAssay(object) = "PC"
object

meta = object@meta.data
colnames(meta)

In [ ]:
table(object$cluster_celltype)

In [ ]:
## function for identifying genes in each cell type with significant interaction effects
find_interaction_genes <- function(object, celltype) {
    print(paste("Processing cell type:", celltype))

    ## subset the object for the specified cell type
    obj_sub = subset(object, subset = cluster_celltype == celltype)
    meta = obj_sub@meta.data

    ## generate pseudo-bulk object
    pseudo_object <- AggregateExpression(obj_sub, assays = "PC", return.seurat = T, group.by = c("individual_ID","diagnosis"))
    print(pseudo_object)

    ## add the region information to the metadata
    df = unique(meta[,c("individual_ID","regions")])
    id = match(pseudo_object$individual_ID, df$individual_ID)
    pseudo_object$regions = df$regions[id]

    ## Get the metadata for the pseudo_object
    meta_pseudo = pseudo_object@meta.data
    rownames(meta_pseudo) = meta_pseudo$individual_ID

    ## Get the counts matrix
    counts = as.matrix(LayerData(pseudo_object,assay = "PC",layer = "counts"))
    counts <- counts[rowSums(counts) > 10, ]  # Filter out genes with zero counts
    colnames(counts) <- paste0(pseudo_object$individual_ID)
    ## Filter the counts matrix to keep only genes expressed in at least 25% of cells
    genes.percent.expression <- rowMeans(counts > 10)

    ## creating the DESeq2 object with brain region as the interaction term
    dds <- DESeqDataSetFromMatrix(countData = counts, 
                                colData = meta_pseudo, 
                                design = ~ diagnosis + regions + diagnosis:regions)
    print(design(dds))
    dds = estimateSizeFactors(dds)
    dds <- DESeq(dds)

    ## fit the reduced model without the interaction term
    dds_lrt <- DESeq(dds, test="LRT", reduced = ~ diagnosis + regions)

    ## get the results for the interaction term
    results(dds_lrt,alpha = 0.05,tidy = T)%>%
        arrange(padj) %>%
        filter(padj < 0.05) %>%
        mutate(gene = row) %>%
        mutate(cell.type = celltype) -> res

    print(paste("Found", nrow(res), "significant interaction genes for cell type:", celltype))

    ## return the results
    return(res)
}
       

In [ ]:
## run the function for each cell type
celltypes <- c("Excitatory","Inhibitory","OPC","Astrocyte","Microglia","Oligodendrocyte")

results_list <- pbmclapply(celltypes, find_interaction_genes, object = object, mc.cores = 10)
## Combine the results into a single data frame
res_combined <- do.call(rbind, results_list)
## Save the results to a CSV file
write.csv(res_combined, "./Results/DEG/pseudobulk_DESeq2_interaction_results.csv", row.names = F)

In [ ]:
## show number of significant interaction genes for each cell type
table(res_combined$cell.type)

In [ ]:
## subset the object for the specified cell type
ct_oi = "OPC"
obj_sub = subset(object, subset = cluster_celltype == ct_oi)
meta = obj_sub@meta.data

## generate pseudo-bulk object
pseudo_object <- AggregateExpression(obj_sub, assays = "PC", return.seurat = T, group.by = c("individual_ID","diagnosis"))
print(pseudo_object)

## add the region information to the metadata
df = unique(meta[,c("individual_ID","regions")])
id = match(pseudo_object$individual_ID, df$individual_ID)
pseudo_object$regions = df$regions[id]

## Get the metadata for the pseudo_object
meta_pseudo = pseudo_object@meta.data
rownames(meta_pseudo) = meta_pseudo$individual_ID

## Get the counts matrix
counts = as.matrix(LayerData(pseudo_object,assay = "PC",layer = "counts"))
counts <- counts[rowSums(counts) > 10, ]  # Filter out genes with zero counts
colnames(counts) <- paste0(pseudo_object$individual_ID)
## Filter the counts matrix to keep only genes expressed in at least 25% of cells
genes.percent.expression <- rowMeans(counts > 10)

## creating the DESeq2 object with brain region as the interaction term
dds <- DESeqDataSetFromMatrix(countData = counts, 
                            colData = meta_pseudo, 
                            design = ~ diagnosis + regions + diagnosis:regions)
print(design(dds))
dds = estimateSizeFactors(dds)
dds <- DESeq(dds)

In [ ]:
## Given a dds object, plot a dot plot of one selected gene across all samples, grouped by brain region, color-coded by diagnosis
plot_gene_expression <- function(dds, gene) {
    # Extract normalized counts for the gene
    norm_counts <- counts(dds, normalized = TRUE)[gene, ]
    
    # Create a data frame for plotting
    df <- data.frame(
        individual_ID = names(norm_counts),
        expression = norm_counts,
        diagnosis = dds$diagnosis,
        regions = dds$regions
    )

    mypal   <- scale_colour_manual(name = "",  values = c("#377EB8","#E41A1C"))
    # Create the dot plot
    ggplot(df, aes(x = regions, y = expression, color = diagnosis)) +
        geom_point(size = 3) +
        labs(title = paste("Expression of", gene),
             x = "Brain Region",
             y = "Normalized Expression") +
        theme_minimal() +
        theme(axis.text.x = element_text(angle = 45, hjust = 1),
            panel.border = element_rect(color = "black", fill = NA, size = 1)
             ) + 
        mypal
}

In [ ]:
res_combined %>%
    group_by(cell.type) %>%
    arrange(padj) %>%
    slice_head(n = 2) %>%
    ungroup()

In [ ]:
## Excitatory cell example

p1 =plot_gene_expression(dds, "VIPR1")
p2 =plot_gene_expression(dds, "NEUROD2")

In [ ]:
## Inhibitory cell example
p3 =plot_gene_expression(dds, "SLC26A4")
p4 =plot_gene_expression(dds, "MPPED1")

In [ ]:
## Astrocyte cell example
p5 =plot_gene_expression(dds, "FREM2")
p6 =plot_gene_expression(dds, "INPP5D")

In [ ]:
## OPC cell example
p7 =plot_gene_expression(dds, "AKR1C3")
p8 =plot_gene_expression(dds, "HAPLN1")

In [ ]:
library(patchwork)

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 12)
p11 = wrap_plots(p1, p2, p3, p4, p5, p6, p7, p8, ncol = 4) +
            plot_annotation(title = "Significant Interaction Genes",
            theme = theme(plot.title = element_text(hjust = 0.5)))

In [ ]:
## Run enrichment analysis for the significant interaction genes
library(clusterProfiler)
library(org.Hs.eg.db)

In [ ]:
res_combined$gene_id <- mapIds(
  # Replace with annotation package for the organism relevant to your data
  org.Hs.eg.db,
  # The vector of gene identifiers we want to map
  keys = res_combined$gene,
  # Replace with the type of gene identifiers in your data
  keytype = "SYMBOL",
  # Replace with the type of gene identifiers you would like to map to
  column = "ENTREZID",
  # In the case of 1:many mappings, return the
  # first one. This is default behavior!
  multiVals = "first"
)

In [ ]:
res_ex <- enrichGO(gene = res_combined[res_combined$cell.type == "Excitatory",]$gene_id,OrgDb= org.Hs.eg.db,ont= c("BP"),pAdjustMethod= "BH",pvalueCutoff= 0.05,readable= TRUE)
res_ex = simplify(res_ex)

res_in <- enrichGO(gene = res_combined[res_combined$cell.type == "Inhibitory",]$gene_id,OrgDb= org.Hs.eg.db,ont= c("BP"),pAdjustMethod= "BH",pvalueCutoff= 0.05,readable= TRUE)
res_in = simplify(res_in)

res_ast <- enrichGO(gene = res_combined[res_combined$cell.type == "Astrocyte",]$gene_id,OrgDb= org.Hs.eg.db,ont= c("BP"),pAdjustMethod= "BH",pvalueCutoff= 0.05,readable= TRUE)
res_ast = simplify(res_ast)

In [ ]:
top_terms <- res_ex@result %>%
  filter(p.adjust < 0.05) %>%
  arrange(p.adjust) %>%
  slice_head(n = 5)

# Ensure the Description is treated as a factor and ordered
top_terms$Description <- factor(top_terms$Description, levels = rev(top_terms$Description))

# Plot the bar plot
options(repr.plot.width = 10, repr.plot.height = 8)
p9 = ggplot(top_terms, aes(x = Description, y = -log10(p.adjust))) +
        geom_bar(stat = "identity", fill = "#33A65C") +
        coord_flip() +
        labs(
          x = "GO Biological Process",
          y = expression(-log[10]~adjusted~p~value),
          title = "Top 5 enriched GOBP terms in Excitatory neurons"
        ) +
        theme_minimal() + 
        theme(panel.border = element_rect(color = "black", fill = NA, size = 1),
            plot.title = element_text(size = 16, face = "bold"),
            axis.title = element_text(size = 14),
            axis.text = element_text(size = 12)
            )
## Inhibitory cell example
top_terms <- res_in@result %>%
  filter(p.adjust < 0.05) %>%
  arrange(p.adjust) %>%
  slice_head(n = 5)

# Ensure the Description is treated as a factor and ordered
top_terms$Description <- factor(top_terms$Description, levels = rev(top_terms$Description))

# Plot the bar plot
options(repr.plot.width = 10, repr.plot.height = 8)
p10 = ggplot(top_terms, aes(x = Description, y = -log10(p.adjust))) +
        geom_bar(stat = "identity", fill = "#23767C") +
        coord_flip() +
        labs(
          x = "GO Biological Process",
          y = expression(-log[10]~adjusted~p~value),
          title = "Top 5 enriched GOBP terms in Excitatory neurons"
        ) +
        theme_minimal() + 
        theme(panel.border = element_rect(color = "black", fill = NA, size = 1),
            plot.title = element_text(size = 16, face = "bold"),
            axis.title = element_text(size = 14),
            axis.text = element_text(size = 12)
            )
        

        

In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)
p12 = wrap_plots(p9,p10,ncol = 2)

In [ ]:
design = c("AAAA
           AAAA
           BBBB")

In [ ]:
ggsave(filename = "./Results/DEG/Interaction_top_gene.pdf",
       p11,
       width = 12, height = 8,dpi = 300)
ggsave(filename = "./Results/DEG/Interaction_top_gene_enrichment.pdf",
       p12,
       width = 16, height = 6,dpi = 300)